# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read csv
cities = "output_data/cities.csv"
city_df = pd.read_csv(cities)
city_df.head(1)

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%)
0,Kapaa,US,22.08,-159.32,82.4,69,12.75,20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = city_df[["Latitude", "Longitude"]]

# Store humidity
humidity = city_df["Humidity (%)"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the DataFrame to find your ideal weather condition. 
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
good_weather_df = city_df.loc[(city_df["Temperature (F)"]<=80) & 
                             (city_df["Temperature (F)"]>=70) &
                             (city_df["Wind Speed (mph)"]<=10) &
                             (city_df["Cloudiness (%)"]==0)].dropna()
good_weather_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%)
80,Porto belo,BR,-27.16,-48.55,71.01,88,2.24,0
104,Arraial do cabo,BR,-22.97,-42.03,73.40,78,8.05,0
107,Quballat,TN,36.54,9.66,75.20,78,1.12,0
117,Caravelas,BR,-17.71,-39.25,71.92,83,7.94,0
123,Lubao,CD,-5.37,25.75,74.53,82,6.67,0
167,Noshiro,JP,40.20,140.02,78.12,82,3.22,0
287,Kamarion,GR,36.80,25.82,78.01,69,3.00,0
314,Roebourne,AU,-20.78,117.13,74.80,39,7.18,0
339,Diego de almagro,CL,-26.37,-70.05,71.10,21,7.90,0
434,Ende,ID,-8.84,121.66,74.52,60,3.96,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# create dataframe for hotel list
hotel_df = good_weather_df.loc[:,["City","Country","Latitude","Longitude"]]
hotel_df["Hotel Name"] = ""
hotel_df.head(1)

,City,Country,Latitude,Longitude,Hotel Name
80,Porto belo,BR,-27.16,-48.55,


In [6]:
# find the first hotel for each city located within 5000 meters of the coordinates
# set parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "lodging",
    "keywork":"hotel",
    "language":"en",
    "radius":5000,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{latitude},{longitude}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"First hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
#     # Wait 1 sec to make another api request to avoid SSL Error
#     time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 80: Porto belo.
First hotel in Porto belo is Costa Verde Beach Hotel.
------------
Retrieving Results for Index 104: Arraial do cabo.
First hotel in Arraial do cabo is Pousada Porto Praia.
------------
Retrieving Results for Index 107: Quballat.
First hotel in Quballat is Firma Ben Ammar.
------------
Retrieving Results for Index 117: Caravelas.
First hotel in Caravelas is Pousada dos Navegantes.
------------
Retrieving Results for Index 123: Lubao.
First hotel in Lubao is Hotel Romania.
------------
Retrieving Results for Index 167: Noshiro.
First hotel in Noshiro is Azuma Ryokan.
------------
Retrieving Results for Index 287: Kamarion.
First hotel in Kamarion is Minoa Hotel Amorgos.
------------
Retrieving Results for Index 314: Roebourne.
First hotel in Roebourne is Harding River Caravan Park.
------------
Retrieving Results for Index 339: Diego de almagro.
First hotel in Diego de almagro is hotel luna del desierto.
------------
Retrieving Results for In

In [7]:
# display hotel_df with hotel names
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
80,Porto belo,BR,-27.16,-48.55,Costa Verde Beach Hotel
104,Arraial do cabo,BR,-22.97,-42.03,Pousada Porto Praia
107,Quballat,TN,36.54,9.66,Firma Ben Ammar
117,Caravelas,BR,-17.71,-39.25,Pousada dos Navegantes
123,Lubao,CD,-5.37,25.75,Hotel Romania
167,Noshiro,JP,40.20,140.02,Azuma Ryokan
287,Kamarion,GR,36.80,25.82,Minoa Hotel Amorgos
314,Roebourne,AU,-20.78,117.13,Harding River Caravan Park
339,Diego de almagro,CL,-26.37,-70.05,hotel luna del desierto
434,Ende,ID,-8.84,121.66,Dwi Putra Hotel - Ende


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))